# Fruit recognition

## Chargement des modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as im
import sklearn as sk
import glob
import os

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

## Chargement des données (train)

In [ ]:
tabImg=pd.DataFrame()
categories = ['Apple Braeburn', 'Apple Granny Smith', 'Apricot', 'Avocado', 'Banana', 'Blueberry', 'Cactus fruit', 'Cantaloupe', 'Cherry', 'Clementine', 'Corn', 'Cucumber Ripe', 'Grape Blue', 'Kiwi', 'Lemon', 'Limes', 'Mango', 'Onion White', 'Orange', 'Papaya', 'Passion Fruit', 'Peach', 'Pear', 'Pepper Green', 'Pepper Red', 'Pineapple', 'Plum', 'Pomegranate', 'Potato Red', 'Raspberry', 'Strawberry', 'Tomato', 'Watermelon']
label = []
root_dir=os.getcwd()+"\\data\\train\\"

for filename in glob.iglob(root_dir+'**\\*.jpg',recursive=True):
    img=im.imread(filename)
    img=img.reshape(img.shape[0]*img.shape[1]*img.shape[2])
    
    fruit_name=filename.split('\\')[-2]
    
    label.append(categories.index(fruit_name))
    tabImg=tabImg.append(pd.Series(img),ignore_index=True)
tabImg['label']=label

In [ ]:
tabImg.shape

In [ ]:
tabImg.to_csv('train.csv',index=False)

In [ ]:
np.save('train.npy', tabImg)

In [ ]:
tabImg.head(5)

## Récupération des données (train)

In [ ]:
#tab=pd.read_csv('train.csv')
tab = np.load('train.npy', allow_pickle=True)

In [ ]:
print(tab.shape)
tab=pd.DataFrame(tab)

In [ ]:
tab.head(5)

In [ ]:
tab=tab.sample(frac=1) # Pour mélanger
tab.head(5)

In [ ]:
tab.isna().sum()

# Traitement des données et modèles

In [ ]:
X=tab.iloc[:,:-1]
print(X.shape)
X.head(5)

In [ ]:
Y=tab.iloc[:,-1]
print(Y.shape)
Y.head(5)

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y,test_size=0.25, random_state=0)

In [ ]:
print(Xtrain.shape)
print(Xtest.shape)
print(ytrain.shape)
print(ytest.shape)

In [ ]:
KKN = KNeighborsClassifier()
clf=KKN.fit(Xtrain,ytrain)

ypredit=clf.predict(Xtest)

In [ ]:
print(accuracy_score(ytest,ypredit))

In [ ]:
print(metrics.confusion_matrix(ytest, ypredit))